In [ ]:
import openai
import pandas as pd
import time

In [ ]:
# OpenAI API 키를 설정합니다.
openai.api_key = 'YOUR_KEY'

In [ ]:
#excel 파일명 list
excel_fname = []
excel_fname = ['your dataset']

In [ ]:
prompt = "You are a knowledgeable pharmacist. Answer the questions about medications accurately. Do not use phrases like ' consult with a doctor or pharmacist.' Please answer in English."

In [ ]:
'''
api_call : 
    요약        :   gpt api에 질문을 하는 함수
    특이사항    :   Timeout이나 server overload 문제 발생시 에러처리해서 다음 질문에 대한 호출 진행.
                    만약 에러 발생시 retry 횟수만큼 다시 질문 시도. 질문 시도 횟수는 retry_cnt로 확인
    prompt      :   You are a knowledgeable pharmacist. 
                    Answer the questions about medications accurately. 
                    Do not use phrases like ' consult with a doctor or pharmacist.' 
                    Please respond in Korean.
    매개변수    :   
                    question : 질문
                    num      : 몇번째 질문인지 확인하기위한 카운터
                    retry    : 오류 발생시 재시도 횟수
    반환값      :
                    문자열과 실수(답변과 걸린 시간)
                    정상적으로 response 받은 경우에는 답변과 걸린 시간을 반환
                    비정상 종료시 "error" 문자열과 걸린 시간 반환
'''


def api_call(question,num,retry):
    retry_cnt = 0
    while retry_cnt < retry:
        try:
            start_time = time.time()  # API 호출 전 시간 기록
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",  # 모델을 지정
                messages=[              # 질문을 입력
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": question}
                ],  
                max_tokens=300,  # 최대 토큰 수
                                # 해당 토큰을 만나면 중지,
                request_timeout = timeout
            )
            end_time = time.time()  # API 호출 후 시간 기록
            time_taken = end_time - start_time  # 걸린 시간 계산
            print("Q",num,": "+ question + " answered : ",time_taken)
            return response.choices[0].message['content'], time_taken
        #에외처리
        except Exception as e:
            end_time = time.time()  # API 호출 후 시간 기록
            time_taken = end_time - start_time  # 걸린 시간 계산
            print(f"An error occured on attempt {retry_cnt + 1} : {e}", time_taken)
            retry_cnt = retry_cnt + 1
            time.sleep(1)    #오류 발생시 1초 대기후 다시 질문
    return "Failed to get an answer", time_taken

In [ ]:
#parameters
#오류 발생시 재시도 횟수
retry = 5
#타임아웃 시간 설정
timeout = 40

In [ ]:
#파일마다 api를 호출해서 답변을 받는다.
for fname in excel_fname:
    #엑셀 파일 pandas dataframe 변환
    df = pd.read_excel(fname)
    #question list에 질문 저장
    questions = df['crude_question']
    answers = []
    time_taken_list = []
    num = 0
    total_time = 0.0
    #질문에 대한 답변을 받아서 answers list에 저장.
    for question in questions:
        answer, time_taken = api_call(question,num,retry)
        answers.append(answer)
        total_time = total_time + time_taken
        time_taken_list.append(time_taken)
        num = num+1
    #pandas dataframe에 gpt 답변 추가
    df['gpt-3.5-turbo answer'] = answers
    df['time_taken'] = time_taken_list
    df['total_time'] = total_time
    #dataframe 엑셀 변환
    df.to_excel(fname, index = False)